In [ ]:
import cv2
import shapely

import pandas as pd
import ipywidgets as widgets
import rasterio as rio
import numpy as np

import matplotlib.pyplot as plt

from scipy import signal
from shapely import wkt
from pathlib import Path

from rasterio import features
from tqdm import tqdm
from IPython.display import display, clear_output

In [ ]:
dataset_path = Path('../../datasets/space_net_7/SN7_buildings_train/train')

In [ ]:
df = pd.read_csv('../../datasets/space_net_7/SN7_buildings_train_csvs/csvs/sn7_train_ground_truth_pix.csv')
df.head()

In [ ]:
df['AOI_name'] = df['filename'].apply(lambda x: x.split('_mosaic_')[1])
df.head()

In [ ]:
print('Total number of \033[1mAreas of interest\033[0m in the dataset \033[1m{}\033[0m'.format(len(df.drop_duplicates(['AOI_name']))))
print('Total number of \033[1mImages\033[0m in the dataset \033[1m{}\033[0m'.format(len(df.drop_duplicates(['filename']))))

print('Total number of \033[1munique houses\033[0m in the dataset \033[1m{:,.0f}\033[0m'.format(len(df.drop_duplicates(['AOI_name', 'id']))))
print('Total number of \033[1mpolygons\033[0m in the dataset \033[1m{:,.0f}\033[0m'.format(len(df)))

### Number of unique houses for each Area of Interest

In [ ]:
df.drop_duplicates(['AOI_name', 'id'])['AOI_name'].value_counts()

### Number of images per AOI

In [ ]:
df.drop_duplicates(['filename', 'AOI_name'])[['AOI_name']].value_counts()

In [ ]:
df

In [ ]:
df['area'] = df['geometry'].apply(lambda x: wkt.loads(x).area)

In [ ]:
df['area'].describe(percentiles=[0.25, 0.5, 0.75, 0.95, 0.99])

### Do polygons update over time?

In [ ]:
evolving_groups = {}

for name, group in tqdm(df.groupby(['AOI_name', 'id'])):
    if len(set(np.round(group['area'], 5))) != 1:
        evolving_groups[name] = group

In [ ]:
len(evolving_groups.keys())

In [ ]:
list(evolving_groups.keys())[12]

In [ ]:
evolving_groups[list(evolving_groups.keys())[12]]

### Visualize polygons examples

In [ ]:
def create_scatter(aoi_name, satellite_image):
    polygons = list(df[(df['AOI_name'] == aoi_name)].drop_duplicates('id', keep='last')['geometry'])
    plt.figure(figsize=(14, 14))
    
    if satellite_image:
        file_name = sorted(list(set(df[df['AOI_name'] == aoi_name]['filename'])))[-1]
        image_path = dataset_path / aoi_name / 'images' / '{}.tif'.format(file_name)
    
        r = rio.open(image_path).read()
        r = r.transpose((1,2,0,))

        plt.imshow(r, origin='lower')

    for polygon in polygons:
        xs, ys = zip(*shapely.get_coordinates(wkt.loads(polygon)))
        
        plt.plot(xs,ys) 
    
    plt.show()

In [ ]:
widgets.interact(create_scatter, aoi_name=set(df['AOI_name']), satellite_image=[False, True])

### Create mask

In [ ]:
aoi_name = np.random.choice(list(set(df['AOI_name'])))
aoi_name

In [ ]:
file_name = sorted(list(set(df[df['AOI_name'] == aoi_name]['filename'])))[-1]
image_path = dataset_path / aoi_name / 'images' / '{}.tif'.format(file_name)

polygons = list(df[(df['AOI_name'] == aoi_name) & (df['filename'] == file_name)]['geometry'])

r = rio.open(image_path).read()
r = r.transpose((1,2,0,))

In [ ]:
mask = features.rasterize(((wkt.loads(polygon), 255) for polygon in polygons), out_shape=r.shape[:2])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 14))

axs[0].imshow(r)
axs[1].imshow(mask)

### visualzie crops

In [ ]:
patch_size = 256

In [ ]:
x = np.random.randint(0, r.shape[0] - patch_size)
y = np.random.randint(0, r.shape[1] - patch_size)
x, y

In [ ]:
crop = r[x: x + patch_size, y: y + patch_size]
crop_mask = mask[x: x + patch_size, y: y + patch_size]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 14))

axs[0].imshow(crop)
axs[1].imshow(crop_mask)

### Visualize timelapses

In [ ]:
def get_image_and_mask(image_path, aoi_name, file_name):
    r = rio.open(image_path).read()

    image = r.transpose((1, 2, 0))[:, :, :-1]

    polygons = list(df[(df['AOI_name'] == aoi_name) & (df['filename'] == file_name)]['geometry'])
    mask = features.rasterize(((wkt.loads(polygon), 255) for polygon in polygons), out_shape=image.shape[:2])
    
    return image, mask

In [ ]:
aoi_name = np.random.choice(list(set(df['AOI_name'])))
aoi_name

In [ ]:
file_names = sorted(list(set(df[df['AOI_name'] == aoi_name]['filename'])))

In [ ]:
file_dict = {file_name: get_image_and_mask(dataset_path / aoi_name / 'images_masked' / '{}.tif'.format(file_name), aoi_name, file_name) for file_name in tqdm(file_names)}

In [ ]:
def visualize(index):
    _, _ = plt.subplots(figsize=(18, 18))
    file_name = file_names[index]

    img, mask = file_dict[file_name]

    img_to_viz = np.hstack([img, np.repeat(mask[:, :, None], 3, axis=-1)])

    plt.imshow(img_to_viz)

In [ ]:
widgets.interact(visualize, index=widgets.IntSlider(min=0, max=len(file_names)-1, step=1, value=0))

### Compute differences

In [ ]:
file_name_1, file_name_2 = sorted(np.random.choice(file_names, 2))

In [ ]:
file_name_1, file_name_2 = file_names[0], file_names[-1]

In [ ]:
img_1, mask_1 = file_dict[file_name_1]
img_2, mask_2 = file_dict[file_name_2]

In [ ]:
img_to_viz_1 = np.hstack([img_1, np.repeat(mask_1[:, :, None], 3, axis=-1)])
img_to_viz_2 = np.hstack([img_2, np.repeat(mask_2[:, :, None], 3, axis=-1)])

img_to_viz = np.vstack([img_to_viz_1, img_to_viz_2])
plt.imshow(img_to_viz)

In [ ]:
udm_mask_1 = np.all(img_1 == 0, axis=-1)*255
udm_mask_2 = np.all(img_2 == 0, axis=-1)*255

udm_mask = np.logical_or(udm_mask_1, udm_mask_2)

difference = np.logical_xor(mask_1 * np.logical_not(udm_mask), mask_2 * np.logical_not(udm_mask))

In [ ]:
fig, axs = plt.subplots(figsize=(18, 18))
img_to_viz = np.hstack([img_1, img_2, np.repeat(difference[:, :, None], 3, axis=-1)*255])

plt.imshow(img_to_viz)

### visualzie crops

In [ ]:
patch_size = 256

In [ ]:
x = np.random.randint(0, img_1.shape[0] - patch_size)
y = np.random.randint(0, img_1.shape[1] - patch_size)
x, y

In [ ]:
crop_1 = img_1[x: x + patch_size, y: y + patch_size]
crop_2 = img_2[x: x + patch_size, y: y + patch_size]

crop_difference = difference[x: x + patch_size, y: y + patch_size]

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(14, 14))

axs[0].imshow(crop_1)
axs[1].imshow(crop_2)
axs[2].imshow(crop_difference)

### Difference heatmap

In [ ]:
aux = np.zeros((difference.shape[0], difference.shape[1]))

In [ ]:
aux[:patch_size, :patch_size] = 1

In [ ]:
output = signal.convolve2d(aux, np.ones((patch_size, patch_size)), mode='valid')

In [ ]:
output.shape

In [ ]:
output

In [ ]:
256*256

In [ ]:
65024+256